<a href="https://colab.research.google.com/github/NickAMC/d624-project-2/blob/umer's-branch/Using_Predictive_analytics_to_predict_PH_of_beverages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the required libraries:**

In [2]:
#! pip install tensorflow keras torch torchvision
#! pip install scikit-learn xgboost lightgbm pyearth
! pip install cubist


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.0/582.0 kB 3.0 MB/s eta 0:00:00


In [4]:
pip install keras==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score , mean_squared_error, mean_absolute_error, make_scorer, r2_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.cross_decomposition import PLSRegression
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from cubist import Cubist
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

# **Loading the data set:**

In [6]:
X_train = pd.read_csv("https://raw.githubusercontent.com/NickAMC/NickAMC.github.io/main/DATA_624_S24/project_2/X_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/NickAMC/NickAMC.github.io/main/DATA_624_S24/project_2/X_test.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/NickAMC/NickAMC.github.io/main/DATA_624_S24/project_2/y_train.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/NickAMC/NickAMC.github.io/main/DATA_624_S24/project_2/y_test.csv")


X_train = X_train.values
y_train = y_train['PH'].values
y_test = y_test['PH'].values
X_test = X_test.values



## **Setting up the dataframe for model metrics:**

In [ ]:
metrics_df = pd.DataFrame(columns=['RMSE', 'MAE', 'R_squared'])

# **Modeling:**

## **Linear Regression:**

In [ ]:
param_gridl = {
    'fit_intercept': [True, False],
    'positive': [True, False],
    'copy_X': [True, False]
}
linear_model = LinearRegression()
grid_search = GridSearchCV(estimator=linear_model, param_grid=param_gridl, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'positive': [True, False]},
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

train_mae = -np.mean(grid_search.cv_results_['mean_test_score'])
train_rmse = np.sqrt(-np.mean(grid_search.cv_results_['mean_test_score']))

print("Training RMSE:", train_rmse)
print("Training MAE:", train_mae)

Training RMSE: 0.14221387536832125
Training MAE: 0.02022478634727641


In [ ]:
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'copy_X': True, 'fit_intercept': False, 'positive': False}


In [ ]:
best_model.fit(X_train, y_train)
y_predl = best_model.predict(X_test)
l_rmse = np.sqrt(mean_squared_error(y_test, y_predl))
l_mae = mean_absolute_error(y_test, y_predl)
lr_squared = r2_score(y_test, y_predl)
metrics_df.loc['Linear Regression'] = [l_rmse,l_mae,lr_squared]



## **Partial Least Sqaures:**


In [ ]:
param_gridp = {
    'n_components': list(range(1, 26))
}
pls_model = PLSRegression()
grid_searchp = GridSearchCV(estimator=pls_model, param_grid=param_gridp, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_searchp.fit(X_train, y_train)
best_modelp = grid_searchp.best_estimator_
best_params = grid_searchp.best_params_
print("Best Hyperparameters:", best_params)
best_modelp.fit(X_train, y_train)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Hyperparameters: {'n_components': 7}


PLSRegression(n_components=7)

In [ ]:
y_predp = best_modelp.predict(X_test)
p_rmse = np.sqrt(mean_squared_error(y_test, y_predp))
p_mae = mean_absolute_error(y_test, y_predp)
pr_squared = r2_score(y_test, y_predp)


metrics_df.loc['Partial least Squares'] = [p_rmse,p_mae,pr_squared]




## **Support Vector Machines (SVM):**

In [7]:
param_grids = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.2, 0.5]
}

svm_regressor = SVR()
scorer = make_scorer(mean_squared_error, greater_is_better=False)


grid_searchs = GridSearchCV(estimator=svm_regressor, param_grid=param_grids, cv=5, scoring=scorer, verbose=1)
grid_searchs.fit(X_train, y_train)


best_modelss = grid_searchs.best_estimator_
best_paramss = grid_searchs.best_params_


print("Best Hyperparameters:", best_params)


Fitting 5 folds for each of 18 candidates, totalling 90 fits


NameError: name 'best_params' is not defined

In [ ]:
best_modelss.fit(X_train, y_train)
y_preds = best_modelss.predict(X_test)
s_rmse = np.sqrt(mean_squared_error(y_test, y_preds))
s_mae = mean_absolute_error(y_test, y_preds)
sr_squared = r2_score(y_test, y_preds)

In [ ]:
metrics_df.loc['Support Vector Machines'] = [s_rmse,s_mae,sr_squared]


In [ ]:
print(p_rmse)
print(s_rmse)
print(l_rmse)

0.13683418110117496
0.12386534300134637
0.13685602100680846


## **Random Forest:**

In [ ]:
param_gridr = {
    'n_estimators': [50, 100, 200],      # Number of trees in the forest
    'max_depth': [None, 10, 20],         # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],     # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]        # Minimum number of samples required to be at a leaf node
}

In [ ]:
rf_regressor = RandomForestRegressor(random_state=42)
scorer = make_scorer(mean_squared_error, greater_is_better=False)


grid_searchr = GridSearchCV(estimator=rf_regressor, param_grid=param_gridr, cv=5, scoring=scorer, verbose=1)
grid_searchr.fit(X_train, y_train)


best_modelr = grid_searchr.best_estimator_
best_paramsr = grid_searchr.best_params_

print("Best Hyperparameters:", best_params)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Hyperparameters: {'n_components': 7}


In [ ]:
best_modelr.fit(X_train, y_train)
y_predr = best_modelr.predict(X_test)
r_rmse = np.sqrt(mean_squared_error(y_test, y_predr))
r_mae = mean_absolute_error(y_test, y_predr)
rr_squared = r2_score(y_test, y_predr)

In [ ]:
metrics_df.loc['Random Forest'] = [r_rmse,r_mae,rr_squared]

## **XGBoost:**

In [ ]:
param_gridx = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

In [ ]:
xgb_regressor = XGBRegressor()
scorer = make_scorer(mean_squared_error, greater_is_better=False)
grid_searchx = GridSearchCV(estimator=xgb_regressor, param_grid=param_gridx, cv=5, scoring=scorer, verbose=1)
grid_searchx.fit(X_train, y_train)
best_modelx = grid_searchx.best_estimator_
best_paramsx = grid_searchx.best_params_

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
best_modelx.fit(X_train, y_train)
y_predx = best_modelx.predict(X_test)
x_rmse = np.sqrt(mean_squared_error(y_test, y_predx))
x_mae = mean_absolute_error(y_test, y_predx)
xr_squared = r2_score(y_test, y_predx)

In [ ]:
metrics_df.loc['XGBoost'] = [x_rmse,x_mae,xr_squared]

## **Neural Network:**

In [ ]:
param_gridn = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],  # Size of hidden layers
    'activation': ['relu', 'tanh'],                   # Activation function
    'solver': ['adam', 'sgd'],                        # Solver for weight optimization
    'learning_rate': ['constant', 'adaptive']         # Learning rate schedule
}

In [ ]:

mlp_regressor = MLPRegressor(max_iter=1000, tol=1e-4, solver='adam')
scorer = make_scorer(mean_squared_error, greater_is_better=False)


grid_searchn = GridSearchCV(estimator=mlp_regressor, param_grid=param_gridn, cv=5, scoring=scorer, verbose=1)
grid_searchn.fit(X_train, y_train)

best_modeln = grid_searchn.best_estimator_
best_paramsn = grid_searchn.best_params_

Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [ ]:
print("Best Hyperparameters:", best_paramsn)
y_predn = best_modeln.predict(X_test)

n_rmse = np.sqrt(mean_squared_error(y_test, y_predn))
n_mae = mean_absolute_error(y_test, y_predn)
nr_squared = r2_score(y_test, y_predn)

Best Hyperparameters: {'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'adam'}


In [ ]:
metrics_df.loc['Neural Network'] = [n_rmse,n_mae,nr_squared]

## **Cubist:**



In [ ]:
cubist_model = Cubist()


param_grid = {
    'n_committees': [3, 5, 7],
    'n_rules': [10,100, 500, 1000, 1500],
    'neighbors': [3, 5, 7]
}

In [ ]:
grid_search = GridSearchCV(cubist_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)
best_cubist = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
y_pred = best_cubist.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("Best parameters:", best_params)
print("Mean squared error:", mse)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Cubist was fitted with feature names
  warnings.warn(


Best parameters: {'n_committees': 7, 'n_rules': 100, 'neighbors': 7}
Mean squared error: 0.010736169917427757


In [ ]:
y_predc = best_cubist.predict(X_test)

c_rmse = np.sqrt(mean_squared_error(y_test, y_predc))
c_mae = mean_absolute_error(y_test, y_predc)
c_squared = r2_score(y_test, y_predc)

print(c_squared)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Cubist was fitted with feature names
  warnings.warn(


0.6423174983350521


In [ ]:
metrics_df.loc['Cubist'] = [c_rmse,c_mae,c_squared]

## **Neural Network keras:**

In [ ]:
def create_model(hidden_units=100, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(1))  # Output layer
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
param_gridkrn = {
    'hidden_units': [50, 100, 200],
    'activation': ['relu', 'tanh'],
    'optimizer': ['adam', 'rmsprop']
}

In [ ]:
model = KerasRegressor(build_fn=create_model, verbose=0)
grid_searchkrn = GridSearchCV(estimator=model, param_grid=param_gridkrn, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_resultkrn = grid_searchkrn.fit(X_train, y_train)

<ipython-input-57-38d42514d158>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Best parameters:", grid_result.best_params_)
print("Mean squared error:", mse)

# **Evaluation:**

In [ ]:
metrics_df

,RMSE,MAE,R_squared
Linear Regression,0.136856,0.105798,0.376012
Partial least Squares,0.136834,0.105861,0.376211
Support Vector Machines,0.123865,0.096630,0.488851
Random Forest,0.104344,0.075822,0.637268
XGBoost,0.103316,0.078006,0.644382
Neural Network,0.180549,0.135015,-0.086018
Cubist,0.103615,0.076443,0.642317
